In [36]:
%%writefile C:\Anaconda\Lib\CAMP.py
    
import datetime as dt
import pandas as pd
import numpy as np
import math as math 
import MathsUtilities as MUte

CampConstants = {
    'BasePhyllochron':120,
    'k':-0.19,
    'BaseDVrn2':0.0,
    'VernSatThreshold':1.0,
    'MethalationThreshold':1.0,
    'TreatmentTtDuration':90, 
    'TtEmerge':90,
    'VrnTreatTemp':1.0, 
    'JuvenileHS':1.1,
    'BaseDVrnX':0
}

def LARPTQ(PTQ,maxLAR,minLAR,PTQhf):
    """Calculate the Leaf Appearance Rate at a given PTQ for a genotype of given minLAR and MaxLAR as formualted by Baumont etal 2019 Journal Expt Botany, Equation 3. 
    Args:
        PTQ: PhotoThermal Quatent (mmol PAR m-2 oCd-1)
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    """
    return minLAR+((maxLAR-minLAR)*PTQ)/ (PTQhf+PTQ)

def CalcPTQ(Tt,Pp):
    Radn = Pp * 3600 * 0.000153 * 2  # Hours * s/h * MJ/m2/s * Coeff_PAR->Total 
    Ft = 1
    if Tt < 15:
        Ft = Tt/15
    return (Radn * 2.285 * Ft)/Tt

def CalcdHS(Tt,HS,PTQ,maxLAR,minLAR,PTQhf):
    """Calculate Haun stage
    Args:
        Tt: Thermal time increment
        HS: Current Haun Stage
        PTQ: PhotoThermalQuotient
        maxLAR: Leaf Appearance Rate at PTQ = infinity (oCd-1)
        minLAR: Leaf Appearance rate at PTQ = 0 (oCd-1)
        PTQhf: PTQ half, controls the curvature of response
    Returns:
        Value to increment Haun stage by
    """
    BasePhyllochron = 1/LARPTQ(PTQ,maxLAR,minLAR,PTQhf)
    StageFactor = np.interp(HS,[0,2,3,7,8,11,12],[0.75,0.75,1.0,1.0,1.4,1.4,1.4],False)
    Phyllochron = BasePhyllochron*StageFactor
    return Tt/Phyllochron

def CalcdPPVrn(Pp, baseUR, maxUR, dHS):
    """ Calculate delta of upregulation for photo period (Pp) sensitive genes
    
    Args:
        Pp: Photoperiod
        baseUR: dVrn/HS below 8h Pp
        maxUR: dVrn/HS above 16h Pp
        dHS: delta haun stage
    Returns:
        delta Vrn representing the additional expression of that gene today
    """
    if Pp <= 8.0:
        return baseUR * dHS
    if (Pp > 8.0) and (Pp < 16.0):
        return (baseUR + (maxUR * (Pp-8)/(16-8))) * dHS
    if (Pp >= 16.0):
        return maxUR * dHS  

def CalcBaseUpRegVrn1(Tt, dHS, BaseDVrn1):
    """ Calculate upregulation of base Vrn1
    
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        BaseDVrn1: coeffociennt for Base Vrn1 expression
        
    Returns:
        delta BaseVrn1 representing the additional Vrn1 expression from base expression
    """
    if Tt < 0: 
        BaseDVrn1 = 0
    return BaseDVrn1 * dHS

def CalcColdUpRegVrn1(Tt,dHS, MaxDVrn1, k):
    """ Upregulation of Vrn1 from cold.  Is additional to base vrn1
        BaseDVrn1 in seperate calculation otherwise te same as Brown etal 2013
    Args:
        Tt: Thermal time increment
        dHS: Delta Haun stage
        MaxDVrn1: coefficient for Maximum upregulation of Vrn1
        
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    UdVrn1 = MaxDVrn1 * np.exp(k*Tt)
    if (Tt < 20):
        return UdVrn1 * dHS
    else:
        return -5

# 
def TSHS(FLN):
    """Haun stage timing of terminal spikelet 
       Inverts equation 5 from Brown etal 2013 FLN =  2.85 + 1.1*TSHS
       Note the intercept differs, was typeo on publication
    Args:
        FLN: The final leaf number
    Returns:
        Estimation of the haun stage timing of terminal spikelet
    """
    return (FLN - 2.85)/1.1

def CAMPmodel(Out, Day, Tt, Pp, Params,
              BasePhyllochron,k,BaseDVrn2,VernSatThreshold,MethalationThreshold,JuvenileHS,BaseDVrnX,TtEmerge):
    """ The Cereal Anthesis Molecular Phenology model.
        Based on the ideas presented in Brown etal 2014 (Annals of Botany)
        Alterations made replacing Vrn4 notion with methalation of Vrn1
        Other alterations to implement working code base
    Args:
        Out: "FLN" or None.  If "FLN" will only return estimated FLN else will return full dataframe with daily state variable values
        Day: List, 1:EndDay representing the timesteps in model run
        Tt: List of same length as Day, representing daily temperature
        Pp: List of same length as Day, representing daily photoperiod
        Params: Series with index {BaseDVrn1, MaxDVrn1, MaxDVrn2, BaseDVrn3} and corresponding values for current cultivar 
        BasePhyllochron: for the given cultivar.  
        k: Temperature response coefficient for Vrn1 response,
        BaseDVrn2: Vrn2 expression below 8h Pp.  
        VernSatThreshold: The amount of Methalated Vrn1 that is required for vernilisation saturation to occur
        MethalationThreshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start
    """
    # Set up Data structure and initialise values
    DF = pd.DataFrame(index = [0], columns = ['Tt','Pp'])
    IsGerminated = False
    IsEmerged = False
    FirstDay = False
    IsMethalating = False
    IsCompetent = False
    IsVernalised = False
    IsInduced = False
    IsReproductive = False
    IsAtFlagLeaf = False
    DF.loc[0,'Tt'] = Tt[0]
    DF.loc[0,'Pp'] = Pp[0]
    DF.loc[0,'PTQ'] = 0
    DF.loc[0,'Stage'] = ""
    DF.loc[0,'HS'] = 0.0
    DF.loc[0,'AccumTt'] = 0
    DF.loc[0,'BaseVrn1'] = 0
    DF.loc[0,'ColdVrn1'] = 0
    DF.loc[0,'MethColdVrn1'] = 0
    DF.loc[0,'Vrn1'] = 0
    DF.loc[0,'Vrn2'] = 0.0
    DF.loc[0,'Vrn3'] = 0.0
    DF.loc[0,'VrnX'] = 0.0
    DF.loc[0,'VSHS'] = 0
    DF.loc[0,'FIHS'] = 0
    DF.loc[0,'TSHS'] = 0
    DF.loc[0,'FLN'] = 2.86
    DF.loc[0,'dHS'] = 0
    DF.loc[0,'dBaseVrn1'] = 0
    DF.loc[0,'dColdVrn1'] = 0
    DF.loc[0,'dMethColdVrn1'] = 0
    DF.loc[0,'pVrn2'] = 0
    DF.loc[0,'dVrn2'] = 0
    DF.loc[0,'dVrn3'] = 0
    DF.loc[0,'dVrnX'] = 0
    d = 0
    
    # Model daily loop
    while (IsAtFlagLeaf == False) and (d < Day[-1]):
        # Increment day
        d +=1
        #Sow crop on day 1
        if (IsGerminated == False):
            DF.loc[d,'Stage'] = 'Germination'
            IsGerminated = True
        # Set daily environment variables
        DF.loc[d,'Tt'] = Tt[d]
        DF.loc[d,'Pp'] = Pp[d]
        #Zero set deltas    
        DF.loc[d,'dBaseVrn1'] = 0
        DF.loc[d,'dColdVrn1'] = 0
        DF.loc[d,'dMethColdVrn1'] = 0
        DF.loc[d,'dVrn2'] = 0
        DF.loc[d,'dVrn3'] = 0
        DF.loc[d,'dVrnX'] = 0
                
        DF.loc[d,'AccumTt'] = DF.loc[d-1,'AccumTt'] + DF.loc[d,'Tt']
        DF.loc[d,'PTQ'] = CalcPTQ(DF.loc[d,'Tt'],DF.loc[d,'Pp'])
        if IsEmerged==True:
            DF.loc[d,'dHS'] = CalcdHS(DF.loc[d,'Tt'],DF.loc[d-1,'HS'],DF.loc[d-1,'PTQ'],Params.MaxLAR,Params.MinLAR,Params.PTQhf)
        else:
            DF.loc[d,'dHS']  = DF.loc[d,'Tt']/(BasePhyllochron*0.75)
       
        #Work out base, cold induced Vrn1 expression and methalyation until vernalisation is complete
        if (IsVernalised==False): 
            DF.loc[d,'dBaseVrn1'] = CalcBaseUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.BaseDVrn1) 
            DF.loc[d,'dColdVrn1'] = CalcColdUpRegVrn1(DF.loc[d,'Tt'],DF.loc[d,'dHS'],Params.MaxDVrn1,k)
        
            # If Vrn1(base + cold) equals Vrn1Target methalate coldVrn1.  BaseVrn1 is all methalated every day
            if (IsMethalating == True):
                DF.loc[d,'dMethColdVrn1'] = max(0.0,DF.loc[d,'dColdVrn1'])
        
            #Then work out Vrn2 expression 
            DF.loc[d,'dVrn2'] = 0
            DF.loc[d,'dVrnX'] = 0.0
            if (IsEmerged==True):
                #DF.loc[d,'SDDurationFactor'] = max(0.5,DF.loc[d-1,'SDDurationFactor'] - (1-CalcdPPVrn(DF.loc[d,'Pp'], 0, 1, 1))/35) 
                if (FirstDay == False):
                    DF.loc[d,'dVrn2'] = CalcdPPVrn(DF.loc[d,'Pp'], BaseDVrn2, Params.MaxDVrn2, DF.loc[d,'dHS']) #* DF.loc[d,'SDDurationFactor']
                    FirstDay == False
                DF.loc[d,'dVrnX'] = CalcdPPVrn(DF.loc[d,'Pp'], BaseDVrnX, Params.MaxDVrnX, DF.loc[d,'dHS'])

            # Set todays Vrn expression values using deltas just calculated
            DF.loc[d,'pVrn2'] = DF.loc[d-1,'pVrn2'] + DF.loc[d,'dVrn2']
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX'] + DF.loc[d,'dVrnX']
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1'] + DF.loc[d,'dBaseVrn1']
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] + DF.loc[d,'dMethColdVrn1'] 
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1'] + DF.loc[d,'dMethColdVrn1'] \
                                                          + DF.loc[d,'dBaseVrn1'] \
                                                          + DF.loc[d,'dVrnX'] 
            DF.loc[d,'ColdVrn1'] = max(0.0,DF.loc[d-1,'ColdVrn1'] + DF.loc[d,'dColdVrn1'] - DF.loc[d,'dMethColdVrn1'])   
            DF.loc[d,'Vrn2'] =  max(0.0,DF.loc[d,'pVrn2'] - DF.loc[d,'Vrn1'])
                                    
        # Once Vernalisation complete, carry over vern expression values from yesterday
        else:
            DF.loc[d,'BaseVrn1'] = DF.loc[d-1,'BaseVrn1']
            DF.loc[d,'ColdVrn1'] = DF.loc[d-1,'ColdVrn1'] 
            DF.loc[d,'MethColdVrn1'] = DF.loc[d-1,'MethColdVrn1'] 
            DF.loc[d,'Vrn1'] = DF.loc[d-1,'Vrn1']
            DF.loc[d,'Vrn2'] = DF.loc[d-1,'Vrn2']
            DF.loc[d,'VrnX'] = DF.loc[d-1,'VrnX']
            
        #Work out if methalation of cold response has started 
        if DF.loc[d,'ColdVrn1']>= MethalationThreshold:
            IsMethalating = True
        else:
            IsMethalating = False

        #work out if vernalisation is complete    
        if (DF.loc[d,'Vrn1'] >= VernSatThreshold) and (DF.loc[d,'Vrn2'] == 0) and (IsVernalised==False):
            IsVernalised  = True
            DF.loc[d,'Stage'] = 'Vern Sat'
        
        #Then work out Vrn3 expression
        if (IsVernalised == True) and (IsReproductive == False):
            DF.loc[d,'dVrn3'] = CalcdPPVrn(DF.loc[d,'Pp'], Params.BaseDVrn3, Params.MaxDVrn3, DF.loc[d,'dHS'])
        DF.loc[d,'Vrn3'] = min(1,DF.loc[d-1,'Vrn3'] + DF.loc[d,'dVrn3'])

        #Then work out phase progression based on Vrn expression
        if (DF.loc[d,'Vrn3'] >= 0.3) and (IsInduced == False):
            IsInduced = True;
        if (DF.loc[d,'Vrn3'] >=  1.0) and (IsReproductive == False):
            IsReproductive = True;
            DF.loc[d,'Stage'] = 'Term Spike'
        
        #Finally do Leaf appearance calculations   
        if (DF.loc[d,'AccumTt'] >= TtEmerge) and (IsEmerged==False):
            IsEmerged = True
            DF.loc[d,'Stage'] = 'Emergence'
            DF.loc[d,'pVrn2'] = Params.iVrn2
        if IsEmerged == True:
            DF.loc[d,'HS'] = DF.loc[d-1,'HS'] +  DF.loc[d-1,'dHS']
        else:
            DF.loc[d,'HS'] = 0
    
        #Set Haun stage variables
        if IsVernalised == False:
            DF.loc[d,'VSHS'] = DF.loc[d,'HS']
        else:
            DF.loc[d,'VSHS'] = DF.loc[d-1,'VSHS']
        if IsInduced == False:
            DF.loc[d,'FIHS'] = DF.loc[d,'HS']
        else: 
            DF.loc[d,'FIHS'] = DF.loc[d-1,'FIHS']
        if IsReproductive == False:
            DF.loc[d,'TSHS'] = DF.loc[d,'HS']
            DF.loc[d,'FLN'] = 2.86 + 1.1 * DF.loc[d,'TSHS']
        else:
            DF.loc[d,'TSHS'] = DF.loc[d-1,'TSHS']
            DF.loc[d,'FLN'] = DF.loc[d-1,'FLN']
       
        #Work out if Flag leaf has appeared.
        if DF.loc[d,'HS'] >= DF.loc[d,'FLN']:
            IsAtFlagLeaf = True    
            DF.loc[d,'Stage'] = 'FlagLeaf'
    
        #Add states to dataframe
        DF.loc[d,'Vrn1'] = DF.loc[d,'Vrn1']
        DF.loc[d,'IsGerminated'] = IsGerminated
        DF.loc[d,'IsEmerged'] = IsEmerged
        DF.loc[d,'IsMethalating'] = IsMethalating
        DF.loc[d,'IsVernalised'] = IsVernalised
        DF.loc[d,'IsInduced'] = IsInduced
        DF.loc[d,'IsReproductive'] = IsReproductive
        DF.loc[d,'IsAtFlagLeaf'] = IsAtFlagLeaf
    
    if (Out == 'FLN') and (IsAtFlagLeaf == True):
        return DF.iloc[-1,:]['FLN']
    else:
        return DF
    
#List of parameter names that CalcCultivarVrnCoeffs calculates

CultivarParamList = ['FLN_LV', 'FLN_LN', 'FLN_SV', 'FLN_SN',
                     'TSHS_LV','TSHS_LN', 'TSHS_SV', 'TSHS_SN', 
                     'VSHS_LV','VSHS_LN', 'VSHS_SV', 'VSHS_SN', 
                     'VernTreatHS','EmergHS',
                     'PPS','MinHSVsTs','MaxDVrn3','BaseDVrn3',                         
                     'BaseDVrn1','MaxVrn2',
                     'BaseVrn1AtVSHS_LN','MaxDVrnX',
                     'BaseVrn1AtVSHS_SV','MethVern1AtVSHS_SV','ColdVrn1AtVSHS_SV','HSColdEffect',
                     'DVrn1AtTt','MaxDVrn1']


def CalcCultivarVrnCoeffs(FLN_LV, FLN_LN, FLN_SV, FLN_SN,
                          TreatmentTtDuration, BasePhyllochron,Tt, k, TtEmerge, VernSatThreshold, MethVrn1Threshold):
    """ Calculate VrnRate paremeters for genotype from given FLN observations
    Args:
        FLN_LV: Final Leaf Number under long day full vernalisation conditions
        FLN_LN: Final Leaf Number under long day un-vernalisation conditions
        FLN_SV: Final Leaf Number under short day full vernalisation conditions
        FLN_SN: Final Leaf Number under short day un-vernalisation conditions
        TreatmentTtDuration: ThermalTime Duration of vernalisation treatment, Default = 90oCd (12 weeks at 1oC)
        BasePhyllochron: For given cultivar
        Tt: Vernalisation Treatment Temperature
        k: Vernalisation temperature response coefficient
        TtEmerge: Thermal time from sowing to emergence
        VernSatThreshold: The amount of Methalated Vrn1 that is required for vernilisation saturation to occur
        MethVrn1Threshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start
    Returns:
        Series with values assigned to each variable in CultivarParameterList 
    """
    #Set up Data Store
    data = pd.Series(index = CultivarParamList)
    
    # Haun stage duration of vernalisation treatment period
    data['VernTreatHS'] = TreatmentTtDuration/(BasePhyllochron*0.75) 
    # Haun stage equivelents from imbibing to Emergence
    data['EmergHS'] = TtEmerge/(BasePhyllochron*0.75)
    # Haun stage at which plant becomes competent to respond to Pp (this is required for Vernalisation)
    JuvenileHS = CampConstants['JuvenileHS']
    
    # Store FLN data
    data['FLN_LV'] = FLN_LV
    data['FLN_LN'] = FLN_LN
    data['FLN_SV'] = FLN_SV
    data['FLN_SN'] = FLN_SN
    
    # Calculate TSHS for each envronment set from FLNData
    data['TSHS_LV']            = TSHS(FLN_LV)
    data['TSHS_LN']            = TSHS(FLN_LN)
    data['TSHS_SV']            = TSHS(FLN_SV)
    data['TSHS_SN']            = TSHS(FLN_SN)
    
    # Haun stage duration from vernalisation saturation to terminal spikelet under long day conditions (MinHSVsTs)
                                 # Assume VS occurs at Competence HS under long Pp full vernalisation for varieties with FLN < 7.3 in these conditions
                                 # Assume maximum of 3, Data from Lincoln CE (CRWT153) showed varieties that harve a high TSHS hit VS ~3HS prior to TS under these conditions
    data['MinHSVsTs']          = min(3,data['TSHS_LV'] - JuvenileHS)
    
    
    # Photoperiod sensitivity (PPS)
                                 # the difference between TSHS at 8 and 16 h pp under 
                                 # full vernalisation.
    data['PPS']                = max(0,data['TSHS_SV'] - data['TSHS_LV'])
    
    # Calculate VSHS for each environment from TSHS and photoperiod response
    data['VSHS_LV']            =  max(JuvenileHS, data['TSHS_LV'] - data['MinHSVsTs'])
    data['VSHS_LN']            =  max(JuvenileHS, data['TSHS_LN'] - data['MinHSVsTs']) 
    data['VSHS_SV']            =  max(JuvenileHS, data['TSHS_SV'] - (data['MinHSVsTs'] + data['PPS']))
    data['VSHS_SN']            =  max(JuvenileHS, data['TSHS_SN'] - (data['MinHSVsTs'] + data['PPS']))
    
    # Maximum delta for Upregulation of Vrn3 (MaxDVrn3)
                                 # Occurs under long Pp conditions
                                 # Assuming Vrn3 increases from 0 - VernSatThreshold between VS to TS and this takes 
                                 # 3 HS under long Pp conditions.
    data['MaxDVrn3']           = VernSatThreshold / data['MinHSVsTs']
    
    # Base delta for upredulation of Vrn3 (BaseDVrn3) 
                                 # Occurs under short Pp conditions
                                 # Assuming Vrn3 infreases from 0 - VernSatThreshold from VS to TS 
                                 # and this take 3 HS plus the additional HS from short Pp delay.
    data['BaseDVrn3']          = VernSatThreshold / (data['MinHSVsTs'] + data['PPS'])
    
    # Base delta for upregulation of Vrn1 (BaseDVrn1) 
                                 # Occurs under non vernalising conditions
                                 # Assuming Vernalisation saturation occurs when Vrn1 == VernSatThreshold  
                                 # under short Pp when no Vrn2 or VrnX are expressed.
                                 # Need to include time from imbib to emerge in duration 
    data['BaseDVrn1']          = VernSatThreshold / (data['VSHS_SN'] + data['EmergHS'])
    
    # Use LV treatment to calculate cold response as this is when vrn2 is expressed so shows the greatest Vrn1 requirement from cold
    # BaseVern1 expressed at vernalisation saturation
    data['BaseVrn1AtVSHS_SV']  = (data['VSHS_SV']+data['EmergHS']) * data['BaseDVrn1']
    
    # Methalated Cold upredulated Vrn1 expression at VSHS
                                 # Subtract out BasedVrn1 expression up to transition
    data['MethVern1AtVSHS_SV'] =  VernSatThreshold - data['BaseVrn1AtVSHS_SV']
    
    # ColdVrn1AtVSHS_SV is the amount of Vrn1 upregulation caused by cold that would be required to give the 
                                 #required MethVern1AtVSHS_SV
    data['ColdVrn1AtVSHS_SV']  = MethVrn1Threshold + data['MethVern1AtVSHS_SV']
    
    # The haun stage duration over which cold temperatures will have an effect.  Will be the minimum of the cold 
    # treatment duration (VernTreatHS) and the HS when vernalisation occurs as cold exposure after this is irrelevent
    data['HSColdEffect']       = min(data['VSHS_SV']+data['EmergHS'], data['VernTreatHS']);
    
    # Cold induced delta upregulation of Vrn1 at treatment temperature ('DVrn1@Tt')
                                 # Methalation of Cold upregulated Vrn1 occurs when 
                                 # ColdUpRegVrn1 > Vrn1Target
                                 # Vrn1Target will be 1.0 in 8 hour conditions
                                 # so ColdUpRegVrn1 at transition will be:
                                 # 1.0 - BaseVern1@Meth + MethColdVrn1@Trans.
                                 # divide by treatment HS duration to give rate
    data['DVrn1AtTt']           = data['ColdVrn1AtVSHS_SV']/data['HSColdEffect']
    
    # Maximum upregulation delta Vrn1 (MUdVrn1)
                                 # The rate of dVrn1/HS at 0oC.  Calculate by rearanging UdVrn1 equation
                                 # UdVrn1 = MUdVrn1 * np.exp(k*Tt) as UdVrn1@Tt is known 
    data['MaxDVrn1']           = data['DVrn1AtTt'] / np.exp(k * Tt) 
    
    # Maximum delta for upregulation of Vrn2 (MaxDVrn2)
                                 # Occurs under long day conditions when Vrn2 expression increases the amount 
                                 # of Vrn1 expression needed to saturate vernalisation and so VS is delayed
                                 # Assume 1 unit of Vrn1 supresses 1 unit of Vrn2.
                                 # So under long days without vernalisation this can be calculated as the 
                                 # amount of baseVrn1 expressed at VS
                                 # any numbers below VernSatThreshold are irrelevent so make that lower bound.
    data['PPVS']               = 1-((data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    data['pVrn2AtVSHS_LV']     = data['MethVern1AtVSHS_SV']  +  data['BaseVrn1AtVSHS_SV']
    data['pVrn2AtVSHS_LN']     = (data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1']
    data['dVrn2_LN']           = data['pVrn2AtVSHS_LN']/data['VSHS_LN']
    data['iVrn2'], \
    data['MaxDVrn2']           = calcVrn2Coeffs(data['PPVS'],
                                                data['pVrn2AtVSHS_LV'],
                                                data['pVrn2AtVSHS_LN'],
                                                data['VSHS_LV'],
                                                data['VSHS_LN'])
    
     # Maximum delta upregulation for VrnX (MaxDVrnX)
                                 # This is an unidentified gene that accellerates vernalisation under long day conditions.
                                 # Only relevent for varieties that have MaxDVrn2 less than 1 (i.e no short day vernalisation response from Vrn2)
                                 # Estimate as the rate of additional Vrn1 expression needed to get to VSHS_LN over and above BaseVrn1 expression a this point
    data['BaseVrn1AtVSHS_LN']   = min(VernSatThreshold,(data['VSHS_LN'] + data['EmergHS']) * data['BaseDVrn1'])
    data['MaxDVrnX']            = 0#(data['MaxVrn2']-data['BaseVrn1AtVSHS_LN'])/(data['VSHS_LN'])
    
    return data     

""" Calculate VrnRate paremeters for genotype from given FLN observations
Args:
    FLN_LV: Final Leaf Number under long day full vernalisation conditions
    FLN_LN: Final Leaf Number under long day un-vernalisation conditions
    FLN_SV: Final Leaf Number under short day full vernalisation conditions
    FLN_SN: Final Leaf Number under short day un-vernalisation conditions
    TreatmentTtDuration: ThermalTime Duration of vernalisation treatment, Default = 90oCd (12 weeks at 1oC)
    BasePhyllochron: For given cultivar
    Tt: Vernalisation Treatment Temperature
    k: Vernalisation temperature response coefficient
    TtEmerge: Thermal time from sowing to emergence
    VernSatThreshold: The amount of Methalated Vrn1 that is required for vernilisation saturation to occur
    MethVrn1Threshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start
Returns:
    Series with values assigned to each variable in CultivarParameterList 
"""
def calcVrn2Coeffs(PPVS,pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN):
    a = 0
    b = 0
    if (PPVS < 0):
        try:
            b = (pVrn2AtVSHS_LN-pVrn2AtVSHS_LV)/(VSHS_LN-VSHS_LV)
            a = 1-VSHS_LV*b
            if a<0.1:
                a,b = forceInt(pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN)
        except:
            a,b = forceInt(pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN)
    return (a,b)

""" Calculate VrnRate paremeters for genotype from given FLN observations
Args:
    FLN_LV: Final Leaf Number under long day full vernalisation conditions
    FLN_LN: Final Leaf Number under long day un-vernalisation conditions
    FLN_SV: Final Leaf Number under short day full vernalisation conditions
    FLN_SN: Final Leaf Number under short day un-vernalisation conditions
    TreatmentTtDuration: ThermalTime Duration of vernalisation treatment, Default = 90oCd (12 weeks at 1oC)
    BasePhyllochron: For given cultivar
    Tt: Vernalisation Treatment Temperature
    k: Vernalisation temperature response coefficient
    TtEmerge: Thermal time from sowing to emergence
    VernSatThreshold: The amount of Methalated Vrn1 that is required for vernilisation saturation to occur
    MethVrn1Threshold: The amount of cold Vrn1 expression required for methalation of Vrn1 to start
Returns:
    Series with values assigned to each variable in CultivarParameterList 
"""
def forceInt(pVrn2AtVSHS_LV,pVrn2AtVSHS_LN,VSHS_LV,VSHS_LN):
    a = 0.1
    b = (((pVrn2AtVSHS_LN+pVrn2AtVSHS_LV)/2)-0.1)/((VSHS_LN+VSHS_LV)/2)
    return (a,b)

Overwriting C:\Anaconda\Lib\CAMP.py
